In [ ]:
import os
os.chdir('./..')

import sys
import time
import pickle

import Methods.common as common
import Methods.commitLoader as commitloader
import Methods.classifier as classifier

try:
    import argparse
    import magic
except ImportError as err:
    print (err)
    sys.exit(-1)

In [ ]:
os.getcwd()

In [ ]:
token_list = list()
token_file = '/mnt/c/Users/User1/Desktop/tokens.txt'

with open(token_file) as f:
    lines = f.readlines()
    for line in lines[0].split(','):
        token_list.append(line)

with open("sorted_dataset_opt.pkl", "rb") as f:
    dataset_dict = pickle.load(f)
    
try:
    common.magic_cookie = magic.open(magic.MAGIC_MIME)
    common.magic_cookie.load()
except AttributeError:
    common.magic_cookie = magic.Magic(mime=True, uncompress=True)
common.verbose_print('[-] initialized magic cookie\n')

In [ ]:
main_dir_results= 'Repos_results/'
repo_dir ='Repos_files/'
repos = 66

common.verbose_mode = False
lenTokens = len(token_list)

for repo_file in range(13,14):
    try:
        variant1 = dataset_dict[repo_file]['source']
        variant2 = dataset_dict[repo_file]['destination']
        
        print(repo_file, ' - ', variant1, ' - ', variant2)
            
        pr_data = common.readPrs(repo_file, variant1)

        ct = 0
        start = time.time()
        result_dict = {}
        for pr_nr in pr_data[0]:
            if int(pr_nr) >=0:
                try:
                    print('Pr_nr: ', pr_nr)
                    destination_sha = pr_data[0][pr_nr]['destination_sha']
                    result_dict[pr_nr] = {}
                    dup_count = 1  

                    parent = pr_data[0][pr_nr]['first_commit_parent']
                    sha = pr_data[0][pr_nr]['last_commit_sha']

                    for files in pr_data[0][pr_nr]['commits_data']:
                        for file in files:
                            result_dict[pr_nr][file] ={}

                            file_ext = commitloader.get_file_type(file)

                            emptyFilePath = 'Empty_files/EmptyFile.txt'

                            if file_ext == 2:
                                emptyFilePath = 'Empty_files/EmptyC.c'
                            elif file_ext == 3:
                                emptyFilePath = 'Empty_files/EmptyJava.java'
                            elif file_ext == 4:
                                emptyFilePath = 'Empty_files/EmptyShell.sh'
                            elif file_ext == 5 or file_ext == 11:
                                emptyFilePath = 'Empty_files/EmptyPython.py'
                            elif file_ext == 6:
                                emptyFilePath = 'Empty_files/EmptyPerl.pl'
                            elif file_ext == 7:
                                emptyFilePath = 'Empty_files/EmptyPHP.php'
                            elif file_ext == 8:
                                emptyFilePath = 'Empty_files/EmptyRuby.rb'
                            elif file_ext == 9:
                                emptyFilePath = 'Empty_files/EmptyYaml.yml'
                            elif file_ext == 10:
                                emptyFilePath = 'Empty_files/EmptyScala.scala'
                            if file_ext == 12:
                                emptyFilePath = 'Empty_files/EmptyJS.js'
                            elif file_ext == 13:
                                emptyFilePath = 'Empty_files/EmptyJson.json'
                            elif file_ext == 14:
                                emptyFilePath = 'Empty_files/EmptyKotlin.kt'
                            elif file_ext == 15:
                                emptyFilePath = 'Empty_files/EmptyXML.xml'
                            elif file_ext == 16:
                                emptyFilePath = 'Empty_files/EmptyGradle.gradle'
                            elif file_ext == 17:
                                emptyFilePath = 'Empty_files/EmptyGemFile.gemfile'


                            if len(files[file]) != 0:
                                try:
                                    common.ngram=4                           

                                    fileName = commitloader.fileName(file)
                                    fileDir = commitloader.fileDir(file)

                                    status = classifier.getStatus(pr_data[0][pr_nr]['commits_data'][0][file])


                                    """
                                        Get the file from the variant2
                                    """
                                    new_file_dir = ''
                                    for h in fileDir:
                                        new_file_dir = new_file_dir + h + '/'
                                    if ct == lenTokens:
                                        ct = 0

                                    destPath, destUrl_ = classifier.getFileFromDest(repo_dir, variant2, destination_sha, repo_file, file, new_file_dir, fileName, file_ext, token_list[ct])
                                    ct += 1

                                    if status =='added':
                                        """
                                            Get the file after the patch from the variant1
                                        """
                                        if ct == lenTokens:
                                            ct = 0
                                        fileAfterPatchDir, fileAfterPatchUrlAdd_= classifier.getFileAfterPatch(repo_dir, variant1, sha, repo_file, pr_nr, file, new_file_dir, fileName, file_ext, token_list[ct])
                                        ct += 1

                                        """
                                            Create the patch file in unified diff format
                                        """
                                        patch_lines = classifier.unified_diff(emptyFilePath, fileAfterPatchDir)
                                        patchPath = repo_dir + str(repo_file) + '/' + variant1 + '/' + str(pr_nr) + '/' + sha + '/patches/' + new_file_dir
                                        patchName = fileName.split('.')[0]
                                        patchPath, dup_count = classifier.save_patch(patchPath, patchName, patch_lines, dup_count)

                                        _class_patch = ''
                                        x_patch_patch, x_patch = classifier.processPatch(patchPath, destPath, 'patch', file_ext)
                                        added = x_patch_patch.added()
                                        match_items_patch = x_patch.match_items()
                                        source_hashes = x_patch.source_hashes()

                                        hunk_matches_patch = classifier.find_hunk_matches_w_important_hash(match_items_patch, 'ED', added, source_hashes, x_patch_patch.items())

                                        hunk_classifications = []
                                        for patch_nr in hunk_matches_patch:
                                            class_buggy =''
                                            class_patch = hunk_matches_patch[patch_nr]['class']

                                            hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                            hunk_classifications.append(hunk_class)

                                        result_mod = {}
                                        result_mod['type'] = 'ADDED'
                                        result_mod['destPath'] = destPath
                                        result_mod['destUrl'] = destUrl_
                                        result_mod['fileAfterPatchUrl'] = fileAfterPatchUrlAdd_
                                        result_mod['fileBeforePatchUrl'] =  ''
                                        result_mod['patchPath'] = patchPath
                                        result_mod['processBuggy'] = ''
                                        result_mod['processPatch'] = x_patch
                                        result_mod['hunkMatchesBuggy'] = ''
                                        result_mod['hunkMatchesPatch'] = hunk_matches_patch
                                        result_mod['patchClass'] = classifier.classify_patch(hunk_classifications)

                                        result_dict[pr_nr][file]['result'] = result_mod

                                    elif status == 'removed':
                                        """
                                            Get the file before the patch from the variant1
                                        """
                                        if ct == lenTokens:
                                            ct = 0
                                        fileBeforePatchDir, fileBeforePatchUrlRem_= classifier.getFileBeforePatch(repo_dir, variant1, sha, parent, repo_file, pr_nr, file, new_file_dir, fileName, file_ext, token_list[ct])
                                        ct += 1

                                        """
                                            Create the patch file in unified diff format
                                        """
                                        patch_lines = classifier.unified_diff(fileBeforePatchDir, emptyFilePath)
                                        patchPath = repo_dir + str(repo_file)+ '/' + variant1 + '/' + str(pr_nr) + '/' + sha + '/patches/' + new_file_dir
                                        patchName = fileName.split('.')[0]
                                        patchPath, dup_count= classifier.save_patch(patchPath, patchName, patch_lines, dup_count)

                                        _class_Buggy = ''
                                        x_buggy_patch, x_buggy = classifier.processPatch(patchPath, destPath, 'buggy', file_ext)
                                        match_items_buggy = x_buggy.match_items()
                                        removed = x_buggy_patch.removed()
                                        source_hashes = x_buggy.source_hashes()

                                        hunk_matches_buggy = classifier.find_hunk_matches_w_important_hash(match_items_buggy, 'MO', removed, source_hashes, x_buggy_patch.items())

                                        hunk_classifications = []
                                        for patch_nr in hunk_matches_buggy:
                                            class_buggy = hunk_matches_buggy[patch_nr]['class']
                                            class_patch = ''

                                            hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                            hunk_classifications.append(hunk_class)

                                        result_mod = {}
                                        result_mod['type'] = 'DELETED'
                                        result_mod['destPath'] = destPath
                                        result_mod['destUrl'] = destUrl_
                                        result_mod['fileAfterPatchUrl'] = ''
                                        result_mod['fileBeforePatchUrl'] =  fileBeforePatchUrlRem_
                                        result_mod['patchPath'] = patchPath
                                        result_mod['processBuggy'] = x_buggy
                                        result_mod['processPatch'] = ''
                                        result_mod['hunkMatchesBuggy'] = hunk_matches_buggy
                                        result_mod['hunkMatchesPatch'] = ''
                                        result_mod['patchClass'] = classifier.classify_patch(hunk_classifications)
                                        result_dict[pr_nr][file]['result'] = result_mod


                                    elif status == 'modified':
                                        """
                                            Get the file before the patch from the variant1
                                        """
                                        if ct == lenTokens:
                                            ct = 0
                                        fileBeforePatchDir, fileBeforePatchUrlMod_ = classifier.getFileBeforePatch(repo_dir, variant1, sha, parent, repo_file, pr_nr, file, new_file_dir, fileName, file_ext, token_list[ct])
                                        ct += 1

                                        """
                                            Get the file after the patch from the variant1
                                        """
                                        if ct == lenTokens:
                                            ct = 0
                                        fileAfterPatchDir, fileAfterPatchUrlMod_ = classifier.getFileAfterPatch(repo_dir, variant1, sha, repo_file, pr_nr, file, new_file_dir, fileName, file_ext, token_list[ct])
                                        ct += 1

                                        """
                                            Create the patch file in unified diff format
                                        """
                                        patch_lines = classifier.unified_diff(fileBeforePatchDir, fileAfterPatchDir)
                                        patchPath = repo_dir + str(repo_file) + '/' + variant1 + '/' + str(pr_nr) + '/' + sha + '/patches/' + new_file_dir
                                        patchName = fileName.split('.')[0]
                                        patchPath, dup_count = classifier.save_patch(patchPath, patchName, patch_lines, dup_count)

                                        """
                                            Compare file before patch to current file for missed opportunities
                                        """
                                        _class_Buggy = ''
                                        x_buggy_patch, x_buggy = classifier.processPatch(patchPath, destPath, 'buggy', file_ext)
                                        match_items_buggy = x_buggy.match_items()
                                        removed = x_buggy_patch.removed()
                                        source_hashes = x_buggy.source_hashes()

                                        hunk_matches_buggy = classifier.find_hunk_matches_w_important_hash(match_items_buggy, 'MO', removed, source_hashes, x_buggy_patch.items())

                                        """
                                            Compare file after patch to current file for effort duplication
                                        """
                                        _class_patch = ''
                                        x_patch_patch, x_patch = classifier.processPatch(patchPath, destPath, 'patch', file_ext)
                                        added = x_patch_patch.added()
                                        match_items_patch = x_patch.match_items()
                                        source_hashes = x_patch.source_hashes()

                                        hunk_matches_patch = classifier.find_hunk_matches_w_important_hash(match_items_patch, 'ED', added, source_hashes, x_patch_patch.items())

                                        if len(hunk_matches_buggy) != len(hunk_matches_patch):
                                            print("Error \n The two sequences are not the same")
                                            print(f"Seq matches buggy has length {len(hunk_matches_buggy)}")
                                            print(f"Seq matches patch has length {len(hunk_matches_patch)}")
    #                                             result_mod = {}
    #                                             result_mod['patchClass'] = 'ERROR'
    #                                             result_mod['processBuggy'] = x_buggy
    #                                             result_mod['processPatch'] = x_patch
    #                                             result_mod['hunkMatchesBuggy'] = hunk_matches_buggy
    #                                             result_mod['hunkMatchesPatch'] = hunk_matches_patch
    #                                             result_dict[pr_nr][file]['result'] = result_mod
    #                                             continue

                                        hunk_classifications = []
                                        if len(match_items_buggy) == 0 or len(match_items_buggy)==0:
                                            if len(hunk_matches_buggy) != 0:
                                                for patch_nr in hunk_matches_buggy:
                                                    class_buggy = hunk_matches_buggy[patch_nr]['class']
                                                    class_patch = 'MC'

                                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                                    hunk_classifications.append(hunk_class)
                                            elif len(hunk_matches_patch) != 0:
                                                for patch_nr in hunk_matches_buggy:
                                                    class_buggy = 'MC'
                                                    class_patch = hunk_matches_patch[patch_nr]['class']

                                                    hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                                    hunk_classifications.append(hunk_class)
                                            else:
                                                class_buggy = 'MC'
                                                class_patch = 'MC'
                                                hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                                hunk_classifications.append(hunk_class)

                                        elif len(hunk_matches_buggy) == len(hunk_matches_patch):
                                            for patch_nr in hunk_matches_buggy:
                                                class_buggy = hunk_matches_buggy[patch_nr]['class']
                                                class_patch = hunk_matches_patch[patch_nr]['class']

                                                hunk_class = classifier.classify_hunk(class_buggy, class_patch)
                                                hunk_classifications.append(hunk_class)

                                        result_mod = {}
                                        result_mod['type'] = 'MODIFIED'
                                        result_mod['destPath'] = destPath
                                        result_mod['destUrl'] = destUrl_
                                        result_mod['fileAfterPatchUrl'] = fileAfterPatchUrlMod_
                                        result_mod['fileBeforePatchUrl'] =  fileBeforePatchUrlMod_
                                        result_mod['patchPath'] = patchPath
                                        result_mod['processBuggy'] = x_buggy
                                        result_mod['processPatch'] = x_patch
                                        result_mod['hunkMatchesBuggy'] = hunk_matches_buggy
                                        result_mod['hunkMatchesPatch'] = hunk_matches_patch
                                        result_mod['patchClass'] = classifier.classify_patch(hunk_classifications)

                                        result_dict[pr_nr][file]['result'] = result_mod
                                    else:
                                        result_mod = {}
                                        result_mod['patchClass'] = 'OTHER EXT'
                                        result_dict[pr_nr][file]['result'] = result_mod

                                except Exception as e:
                                    result_mod = {}
                                    result_mod['patchClass'] = 'ERROR'
                                    result_dict[pr_nr][file]['result'] = result_mod

                                    print('Exception thrown is: ', e)
                                    print('File: ', file)
                            else:
                                result_mod = {}     
                                result_dict[pr_nr][file]['result']=list()
                                if file_ext == 1:
                                    result_mod['patchClass'] = 'OTHER EXT'
                                    result_dict[pr_nr][file]['result'] = result_mod
                                else:
                                    result_mod['patchClass'] = 'NOT EXISTING'
                                    result_dict[pr_nr][file]['result'] = result_mod
                except:
                    continue
        
        end = time.time()
        duration = end-start

        common.pickleFile(main_dir_results + str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_results_to_exe', [result_dict, duration])
    except Exception as e:
        common.pickleFile(main_dir_results + str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_results_to_exe', [result_dict, 0])
        print("Exception: ", e)

In [ ]:
common.pickleFile(main_dir_results + str(repo_file) + '_' + variant1.split('/')[0] + '_' + variant1.split('/')[1] + '_results', [result_dict, 0])